Total Game Score Model - Model Evaluation - GBM

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')
from total_points_model.config import raw_data_file_path
from total_points_model.domain.preprocessing.data_preprocessor import DataPreprocessor
from total_points_model.domain.contracts.mappings import Mappings
from total_points_model.domain.contracts.rolling_columns import RollingColumns
from total_points_model.domain.contracts.modelling_data_contract import ModellingDataContract
from total_points_model.domain.modelling.supermodel import SuperXGBRegressor
from total_points_model.domain.modelling.model_evaluation import XGBRegressorEvaluator

pd.options.display.max_rows = 200
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

New Model

In [ ]:
new_version = 7
model_file_path = '/afl-total-points-model/total_points_model/models/xgb_total_points_v'+str(new_version)+'.joblib'
preproc_file_path = '/afl-total-points-model/total_points_model/preprocessors/preprocessor_v'+str(new_version)+'.joblib'

Comparison Model

In [ ]:
old_version = new_version - 1
compare_train_predictions_file_path = '/afl-total-points-model/total_points_model/predictions/train_predictions_xgb_total_points_v'+str(old_version)+'.csv'
compare_test_predictions_file_path = '/afl-total-points-model/total_points_model/predictions/test_predictions_xgb_total_points_v'+str(old_version)+'.csv'

Load Data

In [ ]:
afl_data = pd.read_csv(raw_data_file_path)
afl_data = afl_data[(afl_data['Year'] > 2004) & (afl_data['Year'] < 2023) & ~(afl_data['Year'] == 2020)]
afl_data.head()

In [ ]:
training_data = afl_data[afl_data[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = afl_data[~afl_data[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [ ]:
X_train, y_train = training_data.drop(columns=[ModellingDataContract.RESPONSE]), training_data[ModellingDataContract.RESPONSE]
X_test, y_test = test_data.drop(columns=[ModellingDataContract.RESPONSE]), test_data[ModellingDataContract.RESPONSE]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Preprocess Data

In [ ]:
preprocessor = joblib.load(preproc_file_path)
preprocessor

In [ ]:
X_train_preproc = preprocessor.transform(X_train)
X_test_preproc = preprocessor.transform(X_test)

In [ ]:
X_train_preproc.tail()

Load Model

In [ ]:
xgb_model = joblib.load(model_file_path)

Get Predictions

In [ ]:
train_preds = xgb_model.predict(X_train_preproc.drop(columns = [ModellingDataContract.ID_COL]))
test_preds = xgb_model.predict(X_test_preproc.drop(columns = [ModellingDataContract.ID_COL]))

Compare Model

In [ ]:
compare_train_data = pd.read_csv(compare_train_predictions_file_path)
compare_test_data = pd.read_csv(compare_test_predictions_file_path)

In [ ]:
compare_train_data = compare_train_data.rename(columns = {'xgb_preds':'old_preds'})
compare_test_data = compare_test_data.rename(columns = {'xgb_preds':'old_preds'})

Evaluate Model

In [ ]:
train_evaluation_data = pd.concat([training_data.reset_index(drop=True), X_train_preproc.reset_index(drop=True)], axis = 1)
train_evaluation_data = train_evaluation_data.loc[:,~train_evaluation_data.columns.duplicated()]
train_evaluation_data['xgb_preds'] = train_preds
train_evaluation_data = pd.merge(train_evaluation_data, compare_train_data[['Match_ID', 'old_preds']], how = "left")
train_evaluation_data.head()

In [ ]:
train_xgb_evals = XGBRegressorEvaluator(model = xgb_model,
                                       data = train_evaluation_data,
                                       actual_name = ModellingDataContract.RESPONSE,
                                       expected_name = "xgb_preds",
                                       compare_name='old_preds'
                                       )

In [ ]:
test_evaluation_data = pd.concat([test_data.reset_index(drop=True), X_test_preproc.reset_index(drop=True)], axis = 1)
test_evaluation_data = test_evaluation_data.loc[:,~test_evaluation_data.columns.duplicated()]
test_evaluation_data['xgb_preds'] = test_preds
test_evaluation_data = pd.merge(test_evaluation_data, compare_test_data[['Match_ID', 'old_preds']], how = "left")
test_evaluation_data.head()

In [ ]:
test_xgb_evals = XGBRegressorEvaluator(model = xgb_model,
                                       data = test_evaluation_data,
                                       actual_name = ModellingDataContract.RESPONSE,
                                       expected_name = "xgb_preds",
                                       compare_name = "old_preds"
                                       )

Model Averages

In [ ]:
train_xgb_evals.data[[ModellingDataContract.RESPONSE, 'xgb_preds']].describe()

In [ ]:
test_xgb_evals.data[[ModellingDataContract.RESPONSE, 'xgb_preds']].describe()

Plots

In [ ]:
train_xgb_evals.plot_distribution()
test_xgb_evals.plot_distribution()

In [ ]:
train_xgb_evals.plot_ave()
test_xgb_evals.plot_ave()

In [ ]:
train_xgb_evals.plot_double_lift_chart()
test_xgb_evals.plot_double_lift_chart()

In [ ]:
train_xgb_evals.plot_feature_ave('Year')
test_xgb_evals.plot_feature_ave('Year')

In [ ]:
train_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")
test_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")

In [ ]:
train_xgb_evals.plot_shap_summary_plot()
test_xgb_evals.plot_shap_summary_plot()

In [ ]:
top_features = test_xgb_evals.get_ranked_feature_importance()
top_10_features = top_features[:10]
top_10_features

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_pdp([col])
    test_xgb_evals.plot_pdp([col])

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_ice([col])
    test_xgb_evals.plot_ice([col])

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_feature_ave(col)
    test_xgb_evals.plot_feature_ave(col)


Model Metrics

In [ ]:
train_xgb_evals.get_mae(), test_xgb_evals.get_mae()

In [ ]:
train_xgb_evals.get_mse(), test_xgb_evals.get_mse()

In [ ]:
train_xgb_evals.get_r2_score(), test_xgb_evals.get_r2_score()

Ad Hoc Checks